## Демонстрационный клон голоса на разных языках

In [8]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

### Инициализация

In [9]:
ckpt_converter = 'checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


В этой демонстрации мы будем использовать OpenAI TTS в качестве базового диктора для создания мультиязычного речевого аудио. Пользователи могут гибко менять базовый динамик в соответствии со своими потребностями. Пожалуйста, создайте файл с именем `.env` и поместите ключ OpenAI как `OPENAI_API_KEY=xxx`. Мы также предоставили модель китайского базового диктора (см. `demo_part1.ipynb`).

In [10]:
import requests
import os
from dotenv import load_dotenv
load_dotenv() 

api_key=os.environ.get("OPENAI_API_KEY")
proxy=os.environ.get("proxy")

In [ ]:
proxies = {
    "http": proxy,
    "https": proxy
}

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

json_data = {
    'model':"tts-1",
    'voice':"nova",
    'input':"This audio will be used to extract the base speaker tone color embedding. " + \
        "Typically a very short audio should be sufficient, but increasing the audio " + \
        "length will also improve the output audio quality."
}


response = requests.post('https://api.openai.com/v1/audio/speech', proxies=proxies, headers=headers, json=json_data)

if response.status_code == 200:
    with open('output.mp3', 'wb') as f:
        f.write(response.content)
    print("MP3 file has been saved as 'output.mp3'")
else:
    print(f"Failed to retrieve MP3 file. Status code: {response.status_code}")

### Получение вкраплений тоновых цветов

`source_se` - это вкрапление цвета тона базового диктора.  Это среднее значение для нескольких предложений с несколькими эмоциями базового диктора. Мы приводим результат непосредственно здесь, но читатели могут самостоятельно извлечь `source_se`.

In [11]:
base_speaker = f"{output_dir}/openai_source_output.mp3"
source_se, audio_name = se_extractor.get_se(base_speaker, tone_color_converter, vad=True)

reference_speaker = 'resources/example_reference.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)

OpenVoice version: v1
[(0.0, 12.48)]
after vad: dur = 12.48


f:\GitProgect\my_openvoice\.venv\lib\site-packages\torch\functional.py:665: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\SpectralOps.cpp:878.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


OpenVoice version: v1
[(0.0, 58.8188125)]
after vad: dur = 58.81798185941043


### Заключение

In [13]:
# Запустите базовый динамик tts
text = [
    "MyShell is a decentralized and comprehensive platform for discovering, creating, and staking AI-native apps.",
    "MyShell هي منصة لامركزية وشاملة لاكتشاف وإنشاء ورهان تطبيقات الذكاء الاصطناعي الأصلية.",
    "MyShell是一个去中心化且全面的平台，用于发现、创建和投资AI原生应用程序。",
    "MyShell एक विकेंद्रीकृत और व्यापक मंच है, जो AI-मूल ऐप्स की खोज, सृजन और स्टेकिंग के लिए है।",
    "MyShellは、AIネイティブアプリの発見、作成、およびステーキングのための分散型かつ包括的なプラットフォームです。",
    "MyShell — это децентрализованная и всеобъемлющая платформа для обнаружения, создания и стейкинга AI-ориентированных приложений."
]
src_path = f'{output_dir}/tmp.wav'

for i, t in enumerate(text):

    json_data = {
    'model':"tts-1",
    'voice':"nova",
    'input':t
    }

    response = requests.post('https://api.openai.com/v1/audio/speech', proxies=proxies, headers=headers, json=json_data)

    if response.status_code == 200:
        with open(src_path, 'wb') as f:
            f.write(response.content)
        print("MP3 file has been saved as 'output.mp3'")
    else:
        print(f"Failed to retrieve MP3 file. Status code: {response.status_code}")


    save_path = f'{output_dir}/output_crosslingual_{i}.wav'

    # Run the tone color converter
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path, 
        src_se=source_se, 
        tgt_se=target_se, 
        output_path=save_path,
        message=encode_message)

MP3 file has been saved as 'output.mp3'
MP3 file has been saved as 'output.mp3'
MP3 file has been saved as 'output.mp3'
MP3 file has been saved as 'output.mp3'
MP3 file has been saved as 'output.mp3'
MP3 file has been saved as 'output.mp3'
